# POS Tagging using RNN

In [1]:
import csv
import time
import spacy
import torch
import random
import numpy as np
from torch import nn
from torchtext.legacy import data
from torchtext.legacy import datasets
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, GloVe
from string import punctuation

In [2]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TEXT = data.Field(lower = True, batch_first = True)
UD_TAGS = data.Field(unk_token = None, batch_first=True)
PTB_TAGS = data.Field(unk_token = None, batch_first=True)
        
fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

TEXT.build_vocab(train_data, 
                 min_freq = 1,
                 vectors = GloVe(name = "42B", dim=300))
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

train_dataloader, val_dataloader, test_dataloader = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [4]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_hiddens, num_layers, num_labels, pad_idx, **kwargs):
        super(BiLSTM, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = pad_idx)
        self.lstm = nn.LSTM(embed_dim, num_hiddens, 
                            num_layers=num_layers, 
                            batch_first=True, 
                            bidirectional=True)
        self.fc = nn.Linear(2 * num_hiddens, num_labels)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        outputs, (h_state, c_state) = self.lstm(embeddings)
        return self.fc(outputs)


In [5]:
VOCAB_SIZE = len(TEXT.vocab)
EMBED_DIM = TEXT.vocab.vectors.shape[1]
NUM_HIDDENS = 64
NUM_LABELS = len(UD_TAGS.vocab)
NUM_LAYERS = 4 
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
EPOCHS = 10 
LR = 5  

model = BiLSTM(VOCAB_SIZE, EMBED_DIM, NUM_HIDDENS, NUM_LAYERS, NUM_LABELS, PAD_IDX)
model.embedding.weight.data.copy_(TEXT.vocab.vectors)
model.to(device)

BiLSTM(
  (embedding): Embedding(16655, 300, padding_idx=1)
  (lstm): LSTM(300, 64, num_layers=4, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=18, bias=True)
)

In [6]:
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = 1, keepdim = True)
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

In [16]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]
criterion = torch.nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 10
    start_time = time.time()

    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()
        text, tags = batch.text, batch.udtags
        predicted_tags = model(text)
        predicted_tags = predicted_tags.view(-1, predicted_tags.shape[-1])
        tags = tags.view(-1)
        loss = criterion(predicted_tags, tags)
        acc = categorical_accuracy(predicted_tags, tags, TAG_PAD_IDX)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += acc.item()
        total_count += 1
        if batch_idx % log_interval == 0 and batch_idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, batch_idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            text, tags = batch.text, batch.udtags
            predicted_tags = model(text)
            predicted_tags = predicted_tags.view(-1, predicted_tags.shape[-1])
            tags = tags.view(-1)
            acc = categorical_accuracy(predicted_tags, tags, TAG_PAD_IDX)
            total_acc += acc.item()
            total_count += 1
    return total_acc/total_count

In [17]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(val_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |    10/   98 batches | accuracy    0.949
| epoch   1 |    20/   98 batches | accuracy    0.944
| epoch   1 |    30/   98 batches | accuracy    0.946
| epoch   1 |    40/   98 batches | accuracy    0.945
| epoch   1 |    50/   98 batches | accuracy    0.947
| epoch   1 |    60/   98 batches | accuracy    0.949
| epoch   1 |    70/   98 batches | accuracy    0.948
| epoch   1 |    80/   98 batches | accuracy    0.947
| epoch   1 |    90/   98 batches | accuracy    0.949
-----------------------------------------------------------
| end of epoch   1 | time:  6.03s | valid accuracy    0.859 
-----------------------------------------------------------
| epoch   2 |    10/   98 batches | accuracy    0.951
| epoch   2 |    20/   98 batches | accuracy    0.952
| epoch   2 |    30/   98 batches | accuracy    0.952
| epoch   2 |    40/   98 batches | accuracy    0.952
| epoch   2 |    50/   98 batches | accuracy    0.952
| epoch   2 |    60/   98 batches | accuracy    0.951
| epoch  

In [18]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

test accuracy    0.880
